In [1]:
#importing the libraries
import pandas as pd
import numpy as np
%pip install matplotlib
import matplotlib.pyplot as plt
%pip install seaborn
import seaborn as sns
%pip install scikit-learn
%pip install tensorflow
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
%pip install lightgbm
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
#  Connect to PostgreSQL & Load Data
DB_USER = "myuser"
DB_PASSWORD = "mypassword"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mydatabase"

connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)

query = 'SELECT * FROM preprocessed_data'
df = pd.read_sql_query(query, con=engine)

print("\n✅ Data loaded successfully!")
print("Dataset Shape:", df.shape)
print("Columns:", df.columns)


✅ Data loaded successfully!
Dataset Shape: (32732, 28)
Columns: Index(['engine_id', 'cycle', 'op_setting_1', 'op_setting_2', 'op_setting_3',
       'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'sensor_6',
       'sensor_7', 'sensor_8', 'sensor_9', 'sensor_10', 'sensor_11',
       'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15', 'sensor_16',
       'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20', 'sensor_21',
       'dataset_name', 'source'],
      dtype='object')


In [4]:
# Compute Remaining Useful Life (RUL)
df['RUL'] = df.groupby('engine_id')['cycle'].transform(max) - df['cycle']

# Optional: Define failure probability (assuming threshold-based classification)
df['failure_probability'] = df['RUL'].apply(lambda x: 1 if x <= 20 else 0)  # Example threshold of 20 cycles

print("\n✅ RUL and Failure Probability computed successfully!")
print(df[['engine_id', 'cycle', 'RUL', 'failure_probability']].head())



✅ RUL and Failure Probability computed successfully!
   engine_id  cycle  RUL  failure_probability
0          1      1  191                    0
1          1      2  190                    0
2          1      3  189                    0
3          1      4  188                    0
4          1      5  187                    0


C:\Users\pc\AppData\Local\Temp\ipykernel_8828\1295633808.py:2: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df['RUL'] = df.groupby('engine_id')['cycle'].transform(max) - df['cycle']


In [5]:
# Feature Engineering
X = df.drop(columns=['RUL', 'failure_probability','dataset_name',"source"], errors='ignore')
y_rul = df['RUL']  # Target for RUL prediction
y_fail = df['failure_probability'] 

# Train-test split (80-20)
X_train, X_test, y_rul_train, y_rul_test = train_test_split(X, y_rul, test_size=0.2, random_state=42)

print("\n✅ Data split into train and test sets.")
print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)


✅ Data split into train and test sets.
Training set shape: (26185, 26)
Test set shape: (6547, 26)


In [11]:
#  SVR
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Random Forest 
rf_rul = RandomForestRegressor(n_estimators=100, random_state=42)
rf_rul.fit(X_train, y_rul_train)

# Train XGBoost 
xgb_rul = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
xgb_rul.fit(X_train, y_rul_train)

# Train SVR
svr_rul = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
svr_rul.fit(X_train_scaled, y_rul_train)

# Train LightGBM 
lgb_rul = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.01, random_state=42)
lgb_rul.fit(X_train, y_rul_train)

print("\n✅ Models trained successfully!")

calculations = {
    "Random Forest (RUL)": {
        "MAE": 0.0,
        "RMSE": 0.0,
        "R² Score": 0.0,
        "Accuracy": 0.0
    },
    "XGBoost (RUL)": {
        "MAE": 0.0,
        "RMSE": 0.0,
        "R² Score": 0.0,
        "Accuracy": 0.0
    },
    "SVR (RUL)": {
        "MAE": 0.0,
        "RMSE": 0.0,
        "R² Score": 0.0,
        "Accuracy": 0.0
    },
    "LightGBM (RUL)": {
        "MAE": 0.0,
        "RMSE": 0.0,
        "R² Score": 0.0,
        "Accuracy": 0.0
    },
}


def regression_accuracy(y_true, y_pred, tolerance=0.1):
    
    within_tolerance = abs(y_true - y_pred) / y_true <= tolerance
    return within_tolerance.mean()

def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    print(f"\n🔹 {model_name} Performance:")
    print(f"  - MAE: {mean_absolute_error(y_test, y_pred):.4f}")
    calculations[model_name]["MAE"] = mean_absolute_error(y_test, y_pred)


    rmse = np.sqrt(mean_squared_error(y_test, y_pred)) 
    print(f"  - RMSE: {rmse:.4f}")
    print(f"  - R² Score: {r2_score(y_test, y_pred):.4f}")
    calculations[model_name]["RMSE"] = rmse
    calculations[model_name]["R² Score"] = r2_score(y_test, y_pred)

    accuracy = regression_accuracy(y_test, y_pred)
    print(f"  - Accuracy (within 10% tolerance): {accuracy:.2%}")
    calculations[model_name]["Accuracy"] = accuracy
    return y_pred

# Evaluate models on test data
y_rul_pred_rf = evaluate_model(rf_rul, X_test, y_rul_test, "Random Forest (RUL)")
y_rul_pred_xgb = evaluate_model(xgb_rul, X_test, y_rul_test, "XGBoost (RUL)")
y_rul_pred_svr = evaluate_model(svr_rul, X_test_scaled, y_rul_test, "SVR (RUL)")
y_rul_pred_lgb = evaluate_model(lgb_rul, X_test, y_rul_test, "LightGBM (RUL)")                      

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001042 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3184
[LightGBM] [Info] Number of data points in the train set: 26185, number of used features: 18
[LightGBM] [Info] Start training from score 119.091083

✅ Models trained successfully!

🔹 Random Forest (RUL) Performance:
  - MAE: 7.7917
  - RMSE: 12.4993
  - R² Score: 0.9672
  - Accuracy (within 10% tolerance): 67.18%

🔹 XGBoost (RUL) Performance:
  - MAE: 5.5723
  - RMSE: 7.6938
  - R² Score: 0.9876
  - Accuracy (within 10% tolerance): 77.46%

🔹 SVR (RUL) Performance:
  - MAE: 27.8765
  - RMSE: 39.7768
  - R² Score: 0.6679
  - Accuracy (within 10% tolerance): 28.00%

🔹 LightGBM (RUL) Performance:
  - MAE: 31.9470
  - RMSE: 40.2227
  - R² Score: 0.6604
  - Accuracy (within 10% tolerance): 24.82%


saving results

In [13]:
import json
from sqlalchemy import create_engine

# Step 5: Define database connection
DB_USER = "myuser"
DB_PASSWORD = "mypassword"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mydatabase"

# Construct the connection string
connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create the SQLAlchemy engine
engine = create_engine(connection_string)


# Step 6: Save preprocessed data to PostgreSQL
table_name = "trainning_data_results"

model_results = [
    {
        "model": "Random Forest (RUL)",
        "calculations": json.dumps({k: float(v) if isinstance(v, np.float64) else v for k, v in calculations["Random Forest (RUL)"].items()}),  # Convert np.float64 to standard float
        "predictions": json.dumps(y_rul_pred_rf.tolist())  # Convert list to JSON
    },
    {
        "model": "XGBoost (RUL)",
        "calculations": json.dumps({k: float(v) if isinstance(v, np.float64) else v for k, v in calculations["XGBoost (RUL)"].items()}),
        "predictions": json.dumps(y_rul_pred_xgb.tolist())
    },
    {
        "model": "SVR (RUL)",
        "calculations": json.dumps({k: float(v) if isinstance(v, np.float64) else v for k, v in calculations["SVR (RUL)"].items()}),
        "predictions": json.dumps(y_rul_pred_svr.tolist())
    },
    {
        "model": "LightGBM (RUL)",
        "calculations": json.dumps({k: float(v) if isinstance(v, np.float64) else v for k, v in calculations["LightGBM (RUL)"].items()}),
        "predictions": json.dumps(y_rul_pred_lgb.tolist())
    }
]
df_model_results = pd.DataFrame(model_results)

# Save the preprocessed data to the database
df_model_results.to_sql(table_name, engine, if_exists="replace", index=False)

print(f"✅ Model results successfully stored in PostgreSQL table '{table_name}'.")

✅ Model results successfully stored in PostgreSQL table 'trainning_data_results'.


With time series

In [6]:
def create_time_series_features(df, window_size=10):
    for i in range(1, 22):  # Loop from sensor_1 to sensor_21
        sensor_col = f'sensor_{i}'
        
        # Rolling Mean & Std
        df[f'rolling_mean_{i}'] = df[sensor_col].rolling(window=window_size).mean()
        df[f'rolling_std_{i}'] = df[sensor_col].rolling(window=window_size).std()
        
        # Lags
        df[f'lag_1_{i}'] = df[sensor_col].shift(1)
        df[f'lag_2_{i}'] = df[sensor_col].shift(2)
        df[f'lag_3_{i}'] = df[sensor_col].shift(3)
        
        # Exponential Weighted Moving Average (EWMA)
        df[f'ewma_{i}'] = df[sensor_col].ewm(span=window_size).mean()
    
    # Interaction Features (Example: sensor_1 * sensor_2)
    df['sensor_1_2_interaction'] = df['sensor_1'] * df['sensor_2']
    
    return df

# Apply feature engineering
data = create_time_series_features(df)

# Drop NaN values caused by rolling & lag features
data.dropna(inplace=True)


C:\Users\pc\AppData\Local\Temp\ipykernel_16556\3274456678.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'ewma_{i}'] = df[sensor_col].ewm(span=window_size).mean()
C:\Users\pc\AppData\Local\Temp\ipykernel_16556\3274456678.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'rolling_mean_{i}'] = df[sensor_col].rolling(window=window_size).mean()
C:\Users\pc\AppData\Local\Temp\ipykernel_16556\3274456678.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many 

In [ ]:
data.head(5)

In [ ]:
y_rul = data['RUL']  # Target for RUL prediction

# Train-test split (80-20)
X_train, X_test, y_rul_train, y_rul_test = train_test_split(X, y_rul, test_size=0.2, random_state=42)



print("\n✅ Data split into train and test sets.")
print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)


In [7]:
# Train Random Forest
rf_rul = RandomForestRegressor(n_estimators=100, random_state=42)
rf_rul.fit(X_train, y_rul_train)

    # Train XGBoost
xgb_rul = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
xgb_rul.fit(X_train, y_rul_train)

    # Train SVR
svr_rul = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
svr_rul.fit(X_train, y_rul_train)

    # Train LightGBM
lgb_rul = LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
lgb_rul.fit(X_train, y_rul_train)

print("\n✅ Models trained successfully!")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004148 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3184
[LightGBM] [Info] Number of data points in the train set: 26185, number of used features: 18
[LightGBM] [Info] Start training from score 119.091083

✅ Models trained successfully!


In [8]:
def regression_accuracy(y_true, y_pred, tolerance=0.1):
    """Calculates percentage of predictions within 10% tolerance."""
    within_tolerance = abs(y_true - y_pred) / y_true <= tolerance
    return within_tolerance.mean()

def evaluate_model(model, X_test, y_test, model_name):
    """Evaluates model performance."""
    y_pred = model.predict(X_test)
    print(f"\n🔹 {model_name} Performance:")
    print(f"  - MAE: {mean_absolute_error(y_test, y_pred):.4f}")
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"  - RMSE: {rmse:.4f}")
    print(f"  - R² Score: {r2_score(y_test, y_pred):.4f}")
    accuracy = regression_accuracy(y_test, y_pred)
    print(f"  - Accuracy (within 10% tolerance): {accuracy:.2%}")
    return y_pred

In [9]:
y_rul_pred_rf = evaluate_model(rf_rul, X_test, y_rul_test, "Random Forest (RUL)")
y_rul_pred_xgb = evaluate_model(xgb_rul, X_test, y_rul_test, "XGBoost (RUL)")
y_rul_pred_svr = evaluate_model(svr_rul, X_test, y_rul_test, "SVR (RUL)")
y_rul_pred_lgb = evaluate_model(lgb_rul, X_test, y_rul_test, "LightGBM (RUL)")


🔹 Random Forest (RUL) Performance:
  - MAE: 7.7917
  - RMSE: 12.4993
  - R² Score: 0.9672
  - Accuracy (within 10% tolerance): 67.18%

🔹 XGBoost (RUL) Performance:
  - MAE: 5.5723
  - RMSE: 7.6938
  - R² Score: 0.9876
  - Accuracy (within 10% tolerance): 77.46%

🔹 SVR (RUL) Performance:
  - MAE: 21.7412
  - RMSE: 36.8363
  - R² Score: 0.7152
  - Accuracy (within 10% tolerance): 51.75%

🔹 LightGBM (RUL) Performance:
  - MAE: 7.4456
  - RMSE: 9.8617
  - R² Score: 0.9796
  - Accuracy (within 10% tolerance): 72.28%


 LSTM for RUL Prediction

In [11]:
df.head()


,engine_id,cycle,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,lag_2_20,lag_3_20,ewma_20,rolling_mean_21,rolling_std_21,lag_1_21,lag_2_21,lag_3_21,ewma_21,sensor_1_2_interaction
9,1,10,-0.0033,0.0001,100.0,0.0,0.170588,0.463756,0.307394,0.0,...,0.643411,0.744186,0.657150,0.689492,0.062431,0.713152,0.579541,0.672512,0.696860,0.0
10,1,11,0.0018,-0.0003,100.0,0.0,0.338235,0.265511,0.310432,0.0,...,0.705426,0.643411,0.649593,0.697801,0.073100,0.800557,0.713152,0.579541,0.720688,0.0
11,1,12,0.0016,0.0002,100.0,0.0,0.273529,0.300188,0.302161,0.0,...,0.627907,0.705426,0.662297,0.689784,0.072741,0.813500,0.800557,0.713152,0.707893,0.0
12,1,13,-0.0019,0.0004,100.0,0.0,0.570588,0.274702,0.313639,0.0,...,0.620155,0.627907,0.652504,0.680251,0.086862,0.656646,0.813500,0.800557,0.673168,0.0
13,1,14,0.0009,0.0000,100.0,0.0,0.358824,0.499478,0.285449,0.0,...,0.713178,0.620155,0.682241,0.681461,0.086751,0.530967,0.656646,0.813500,0.674441,0.0


In [23]:
def create_time_series_data(df, window_size=10):
    X, y = [], []
    for i in range(len(df) - window_size):
        X.append(df.iloc[i : i + window_size, :-1].values)  # Features
        y.append(df.iloc[i + window_size, -1])  # Target (RUL)
    return np.array(X), np.array(y)

# Drop NaN values (after rolling/lag features)
df.dropna(inplace=True)

# Select features and target
features = df.drop(columns=['RUL', 'failure_probability', 'dataset_name'], errors='ignore')
target = df['RUL']

print(features.dtypes)  # Check data types of all columns
print(features.head())  # Look at first few rows

for col in df.columns:
    if df[col].dtype == 'object':  # If column contains strings
        print(f"⚠ Column '{col}' contains non-numeric values.")
        print(df[col].unique())  # Print unique values




engine_id                   int64
cycle                       int64
op_setting_1              float64
op_setting_2              float64
op_setting_3              float64
                           ...   
lag_1_21                  float64
lag_2_21                  float64
lag_3_21                  float64
ewma_21                   float64
sensor_1_2_interaction    float64
Length: 153, dtype: object
    engine_id  cycle  op_setting_1  op_setting_2  op_setting_3  sensor_1  \
9           1     10       -0.0033        0.0001         100.0       0.0   
10          1     11        0.0018       -0.0003         100.0       0.0   
11          1     12        0.0016        0.0002         100.0       0.0   
12          1     13       -0.0019        0.0004         100.0       0.0   
13          1     14        0.0009        0.0000         100.0       0.0   

    sensor_2  sensor_3  sensor_4  sensor_5  ...  lag_2_20  lag_3_20   ewma_20  \
9   0.170588  0.463756  0.307394       0.0  ...  0.643411  0.

In [25]:
# Normalize features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)


In [27]:
# Convert back to DataFrame
df_scaled = pd.DataFrame(features_scaled, columns=features.columns)
df_scaled['RUL'] = target.values  # Add back target

# Create time-series dataset
X, y = create_time_series_data(df_scaled, window_size=5)  # 10 time steps

# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape for LSTM (samples, time_steps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2]))

print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

MemoryError: Unable to allocate 38.2 MiB for an array with shape (153, 32723) and data type float64

In [ ]:
# Define LSTM model
model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1)  # Output layer for RUL prediction
])

# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

# Model summary
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Save model
model.save("lstm_rul_model.h5")

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Make predictions
y_pred = model.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"\n🔹 LSTM Performance:")
print(f"  - MAE: {mae:.4f}")
print(f"  - RMSE: {rmse:.4f}")
print(f"  - R² Score: {r2:.4f}")


In [122]:
# Save trained models
joblib.dump(rf_rul, "rf_rul_model.pkl")
joblib.dump(xgb_rul, "xgb_rul_model.pkl")
if y_fail is not None:
    joblib.dump(rf_fail, "rf_fail_model.pkl")
    joblib.dump(xgb_fail, "xgb_fail_model.pkl")

print("\n✅ Models saved successfully!")

#  Store predictions in PostgreSQL
df_test = X_test.copy()
df_test['RUL_Prediction_RF'] = y_rul_pred_rf
df_test['RUL_Prediction_XGB'] = y_rul_pred_xgb

if y_fail is not None:
    df_test['Failure_Prob_Prediction_RF'] = y_fail_pred_rf
    df_test['Failure_Prob_Prediction_XGB'] = y_fail_pred_xgb

df_test.to_sql("predicted_results", engine, if_exists="replace", index=False)
print("\n✅ Predictions stored in PostgreSQL.")

#  Display sample predictions
print("\n🔍 Sample Predictions:")
print(df_test[['RUL_Prediction_RF', 'RUL_Prediction_XGB']].head())

if y_fail is not None:
    print(df_test[['Failure_Prob_Prediction_RF', 'Failure_Prob_Prediction_XGB']].head())


NameError: name 'xgb_fail' is not defined

In [ ]:
# Define table name
result_table = "ml_predictions"

# Save predictions to PostgreSQL
df_results.to_sql(result_table, engine, if_exists="replace", index=False)

print(f"✅ ML predictions saved to PostgreSQL table '{result_table}'.")
